In [1]:
text: str = None
with open('../data/input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [2]:
print(len(text), 'characters')

1115394 characters


In [3]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
chars: list[str] = sorted(list(set(text)))
vocab_size: int = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
from typing import Callable

stoi: dict[str, int] = { ch: i for i, ch in enumerate(chars) }
itos: dict[int, str] = { i: ch for i, ch in enumerate(chars) }
encode: Callable[[str], list[int]] = lambda s: [stoi[c] for c in s]
decode: Callable[[int], str] = lambda l: ''.join([itos[i] for i in l])

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch
from torch import tensor
data: tensor = tensor(encode(text), dtype=torch.long)
print(data.size(), 'of type', data.dtype)
print(data[:250])

torch.Size([1115394]) of type torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41

In [7]:
# Split into train and validation sets
n: int = int(0.9 * len(data))
train_data: tensor = data[:n]
val_data: tensor = data[n:]

In [8]:
# Maximum sequence length
block_size: int = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
# This one input block x actually encompasses 8 possible training samples.
#
# This is the source of the motivation for using future masking during
# training.
#
# There are two primary reasons this is useful:
# 1) It enables a high degree of computational efficiency when we implement
#    future masking to train the model on multiple variable-length contexts
# 2) Training the model on every possible length of context size means the
#    model gains exposure to context lengths (i.e. prompts) consisting of
#    as few as 1 token, yielding a model capable of reasoning over a wider
#    variety of prompts
x: tensor = train_data[:block_size]
y:tensor =  train_data[1: block_size + 1]
for t in range(block_size):
  context = x[:t + 1]
  target = y[t]
  print(f'when input is {context}, the target is {target}')

when input is tensor([18]), the target is 47
when input is tensor([18, 47]), the target is 56
when input is tensor([18, 47, 56]), the target is 57
when input is tensor([18, 47, 56, 57]), the target is 58
when input is tensor([18, 47, 56, 57, 58]), the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [10]:
torch.manual_seed(1337)
batch_size: int = 4
block_size: int = 8

def get_batch(split: str) -> tuple[tensor, tensor]:
  data: tensor = train_data if split == 'train' else val_data

  # Pick `batch_size` number of random offsets into the dataset
  offsets: tensor = torch.randint(len(data) - block_size, (batch_size,))
  x: tensor = torch.stack([data[i: i + block_size] for i in offsets])
  y: tensor = torch.stack([data[i + 1: i + block_size + 1] for i in offsets])

  return x, y

# The (4, 8) batch `xb` contains 32 independent (as far as the transformer
# is concerned) usable training samples. `yb` comes into use at the very
# end when we compose the loss function.
xb, yb = get_batch('train')
print('inputs:')
print(xb.size())
print(xb)
print('targets:')
print(yb.size())
print(yb)
print('---')

for b in range(batch_size):
  for t in range(block_size):
    context: tensor = xb[b, :t + 1]
    target: tensor = yb[b, t]
    print(f'when input is {context.tolist()}, the target is {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
when input is [24], the target is 43
when input is [24, 43], the target is 58
when input is [24, 43, 58], the target is 5
when input is [24, 43, 58, 5], the target is 57
when input is [24, 43, 58, 5, 57], the target is 1
when input is [24, 43, 58, 5, 57, 1], the target is 46
when input is [24, 43, 58, 5, 57, 1, 46], the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target is 39
when input is [44], the target is 53
when input is [44, 53], the target is 56
when input is [44, 53, 56], the target is 1
when input is [44, 53, 56, 1], the target is 58
when input is [44, 53, 56, 1, 58], 

## Baseline Bigram Model

In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size: int) -> None:
    super().__init__()
    
    # Each token directly reads off the logits for the next token
    # from a lookup table.
    #
    # For example, given input index 25, we will retrieve row
    # 25 from this table, and that row will contain `vocab_size`
    # scores, one score corresponding to each possible subsequent
    # token.
    #
    # From there we can basically just softmax that entire row
    # to get a probability distribution over the likelihood of
    # seeing any given token appear next.
    self.token_embeddings = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx: tensor, targets: tensor = None) -> tuple[tensor]:
    # Let:
    # - B: batch size [4]
    # - T: maximum sequence/block length [8]
    # - C: embedding size/dimensionality [65]

    # idx and targets both have shape (B, T)
    logits: tensor = self.token_embeddings(idx) # (B, T) => (B, T, C)

    loss: tensor = None
    if targets != None:
      # PyTorch cross entropy wants (B, C) shape, so we reshape our
      # tensors to accommodate
      #
      # TODO
      # Try just logits.transpose(1, 2): (B, T, C) => (B, C, T) with
      # targets: (B, T) where each cell in `targets` contains the index
      # of the correct class.
      B, T, C = logits.size()
      logits = logits.view(B * T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss
  
  def generate(self, idx: tensor, max_new_tokens: int) -> tensor:
    # idx: (B, T) tensor of indices in the current context
    for _ in range(max_new_tokens):
      # get predictions
      logits, loss = self(idx)

      # focus only on the last token in the sequence since this is
      # for a bigram model - given a single token, we're trying to
      # predict the next token, thereby generating a bigram.
      #
      # note that this is computationally sort of absurd since this
      # generate function consumes the entire history of generated
      # tokens so far but only ever uses the last generated token.
      #
      # this total history of generated tokens will become useful
      # when we implement a transformer that can attend to all
      # prior generated tokens so we will keep it this way, but
      # just note that it's silly for this particular bigram model.
      logits = logits[:, -1, :] # (B, T, C) => (B, C)

      # convert to probability distribution
      probs = F.softmax(logits, dim=-1) # (B, C) => (B, C)

      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T) => (B, T+1)

    return idx

model: nn.Module = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.size())
print(loss) # Should be approximately equal to -ln(1/65) reflecting random guessing

# We start the generation with a (1, 1) (single-element) tensor with a single 0. Note
# that 0 corresponds to the newline character in our sorted list of tokens, so it's a
# reasonable way to initiate the response sequence.
#
# model.generate() will then append 100 new tokens, yielding a tensor of shape
# (1, 101), which we then index by the 0th row and convert it into a simple python
# list. We then pass our list of integers to our decode function to get a string
# of characters.
idx: tensor = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([1, 101])

pdcbf?pGXepydZJSrF$Jrqt!:wwWSzPNxbjPiD&Q!a;yNt$Kr$o-gC$WSjJqfBKBySKtSKpwNNfyl&w:q-jluBatD$Lj;?yzyUca


## Bigram Model Training

In [24]:
# LR is often 3e-4 for bigger networks but smaller networks like this one
# are fine with much bigger gradient updates like 1e-3
optimizer: torch.optim.AdamW = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [34]:
batch_size: int = 32
for steps in range(10000):
  xb, yb = get_batch('train')

  optimizer.zero_grad(set_to_none=True)
  logits, loss = model(xb, yb)
  loss.backward()
  optimizer.step()

print(loss.item())

2.454188108444214


In [36]:
print(decode(model.generate(idx, max_new_tokens=400)[0].tolist()))




EYod hat ncothave ingr t sou fr in nd songnokerd t ker.
FI ases ratha h cuemaloot Rwakithiorortthe kndit, I
Th'sco ntath! f peso s lt me adws t wnolour!'sherelas ast benan.
A helagriss, s hy nd chaden, dy myoulothoofouris, yard

AUCA:
Whathe ig.
TEd; he s thicrous pof thofaro-
Lonthanerimpud EO:
blaithencken th g wh se had.
Vore tha-otr;
BUELI t WAmble ofe'spthespout glar belos hat itliths s sar
